In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import requests
import urllib.request
from bs4 import BeautifulSoup
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import warnings
from sklearn import metrics

warnings.filterwarnings('ignore')


from sklearn import model_selection
from sklearn.model_selection import cross_val_score


from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

In [2]:
dbname = 'ncaabb'
username = 'gshau' # change this to your username
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))


postgres://gshau@localhost/ncaabb
True


In [14]:

class Tournament:
    def __init__(self,teamList):
        '''Tournament class, takes as arguments:\
          teamList - list of teams in bracket order'''
        self.teamList = np.array(teamList)
        self.nTeams = len(teamList)
#         self.teamDict =


    def getPairings(self,teamList):
        pairings=[]
        for iteam in range(len(teamList)/2):
            pairings.append([teamList[2*iteam],teamList[2*iteam+1]])
        return pairings

    def setModel(self,model):
        self.model = model

    def predictGame(self,game):        
        outcome = model(game[0].properties,game[1].properties)
        return outcome

    def simulate(self,probThr):

        teamList = self.teamList
        nTeamRemain = len(teamList)
        nRound = 1
        teamLists={}
        teamLists[nRound]=teamList
        while nTeamRemain > 1:
            pairs = self.getPairings(teamList)
            teamList=[]
            for p in pairs:
                print('Pair: ',p[0].name, p[1].name,nTeamRemain)
                prob = self.predictGame(p)
                if prob > probThr():
                    teamList.append(p[0])
                else:
                    teamList.append(p[1])
            nRound+=1
            teamLists[nRound]=teamList
            nTeamRemain = len(teamList)

        return teamLists


class Team:
    def __init__(self,name,properties):
        self.name=name
        self.properties=properties


In [4]:

dbname = 'ncaabb'
username = 'gshau' # change this to your username
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))


def loadTourney(year):
    df=pd.read_sql('SELECT * FROM tourney_team_'+str(year), engine,index_col='index')
    return df

def loadTeamData(year):
    df=pd.read_sql('SELECT * FROM team_data_'+str(year), engine,index_col='index')
    return df
    teamList={}
    for r in df.itertuples():
        teamList[r.School]=Team(r.School,dict(r))
    
    return teamList

def getYears():
    df=pd.read_sql("SELECT table_name FROM information_schema.tables WHERE table_schema='public' AND table_type='BASE TABLE';", engine)
    yearList=sorted(np.unique(np.array([int(tn.split('_')[-1]) if 'team' in tn else 0 for tn in df.table_name.values])),reverse=True)[:-1]
    return yearList

def initializeTourney(year=2017):
    teamData=loadTeamData(year)
    teamNamesInTourney = loadTourney(year)
    teamNames=teamNamesInTourney.teamName.values
    teamList=[]
    for teamName in teamNames:
        properties=teamData[teamData.School==teamName]
        teamList.append(Team(teamName,dict(properties)))
    
    
    tourney=Tournament(teamList)
    return tourney


In [9]:
teamData.keys()

Index(['School', 'wpct', 'SRS', 'SOS', 'TmPts', 'OppPts', 'Pace', 'ORtg',
       'FTr', '3PAr', 'TS', 'TRB', 'AST', 'STL', 'BLK', 'eFG', 'TOV', 'ORB',
       'FT/FGA', 'sched_url', 'name', 'fullName', 'Rank', 'Conf', 'WL',
       'AdjEM', 'AdjO', 'AdjD', 'AdjT', 'Luck', 'AdjEM.1', 'OppO', 'OppD',
       'NCSOS_AdjEM'],
      dtype='object')

In [13]:
year=2016
teamData=loadTeamData(year)
teamNamesInTourney = loadTourney(year)
teamNames=teamNamesInTourney.teamName.values
teamList=[]
for teamName in teamNames:
    properties = teamData[teamData.name==teamName]
    propertyDict = dict([(key,properties.iloc[0][key]) for key in properties.keys()])
    teamList.append(Team(teamName,propertyDict))


tourney=Tournament(teamList)
# return tourney


In [ ]:
tourney.simulate

In [111]:
dict([(key,properties.iloc[0,key]) for key in properties.keys()])

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

In [77]:
d=dict(properties.T)
d

{215: School                                         north-dakota
 wpct                                                  0.688
 SRS                                                   -1.83
 SOS                                                   -5.45
 TmPts                                                  2578
 OppPts                                                 2357
 Pace                                                   72.9
 ORtg                                                  108.8
 FTr                                                    0.26
 3PAr                                                  0.305
 TS                                                    0.573
 TRB                                                    50.5
 AST                                                    52.6
 STL                                                    10.6
 BLK                                                     7.5
 eFG                                                   0.542
 TOV               

In [59]:
tourney=initializeTourney(2017)
team=tourney.teamList[2]
team.properties

ValueError: dictionary update sequence element #0 has length 32; 2 is required